In [ ]:
# The following should reload files if we change them on disk
%load_ext autoreload
%aimport econsim.work, econsim.resources, econsim.makeragent, econsim.designer, econsim.producer, econsim.serviceprovider, econsim.utils, econsim.distmakingmodel, econsim.market
%autoreload 1

from pdb import set_trace

In [ ]:
from econsim.utils import generate_ranges
from econsim.distmakingmodel import DistMakingModel

_range_weights, price_weight, quality_ratio, range_living_cost, range_threshold, range_resources = generate_ranges(2)

In [ ]:
graphic_params = {
    "designers": {
        "type": "SliderInt",
        "value": 80,
        "label": "Designers",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "producers": {
        "type": "SliderInt",
        "value": 40,
        "label": "Producers",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "initial_wealth": {
        "type": "SliderInt",
        "value": 10,
        "label": "Initial Wealth",
        "min": 0,
        "max": 100,
        "step": 1
    },
    "living_cost": {
        "type": "SliderFloat",
        "value": 0.1,
        "label": "Cost of Living",
        "min": 0.1,
        "max": max(range_living_cost),
        "step": 0.1
    },
    "resources_amount": {
        "type": "SliderInt",
        "value": min(range_resources),
        "label": "Resources",
        "min": min(range_resources),
        "max": max(range_resources),
        "step": 100
        
    },
    "threshold": {
        "type": "SliderFloat",
        "value": 0.1,
        "label": "Threshold to buy",
        "min": 0.1,
        "max": max(range_threshold),
        "step": 0.1
    },
    "weights": {
        "type": "SliderFloat",
        "value": 0.1,
        "label": "Weight of price",
        "min": 0.1,
        "max": 0.5,
        "step": 0.1
    },
    "quality_ratio": {
        "type": "SliderFloat",
        "value": 0.5,
        "label": "Ratio quality/sustainability",
        "min": 0.1,
        "max": 1,
        "step": 0.1
    }
}



In [ ]:
from econsim.designer import Designer
from econsim.producer import Producer

def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {}

    if type(agent) is Designer:
        portrayal["Shape"] = "econsim/resources/icons8-designer-64.png"
        # https://icons8.com/icon/9APE8Mw8ziYC/designer icon by https://icons8.com
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 1

    elif type(agent) is Producer:
        portrayal["Shape"] = "econsim/resources/icons8-producer-64.png"
        # https://icons8.com/icon/YdrIWceHZShx/producer icon by https://icons8.com
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 2
        portrayal["text"] = round(agent.wealth, 1)
        portrayal["text_color"] = "White"

    elif type(agent) is ServiceProvider:
        if agent.fully_grown:
            portrayal["Color"] = ["#00FF00", "#00CC00", "#009900"]
        else:
            portrayal["Color"] = ["#84e184", "#adebad", "#d6f5d6"]
        portrayal["Shape"] = "rect"
        portrayal["Filled"] = "true"
        portrayal["Layer"] = 0
        portrayal["w"] = 1
        portrayal["h"] = 1

    return portrayal


In [ ]:

# def agent_portrayal(agent):
#     portrayal = {"Shape": "circle", "Filled": "true", "r": 0.5}
#     agtype = DistMakingModel.agent_type(agent.unique_id)
#     if agtype == "Designer":
#         shape = "square"
#         color = "tab:red"
#     elif agtype == "Producer":
#         shape = "circle"
#         color = "tab:blue"
#     else:
#         shape = "triangle"
#         color = "tab:green"
#     size = 20
#     # color = "tab:red"
#     # if agent.wealth > 0:
#     #     size = 50
#     #     color = "tab:blue"
#     # "edge_color": edge_color,
#     # "node_color": node_color,
#     return {"size": size, "color": color, "shape": shape, "edge_color": "tab:red"}


In [ ]:
import solara
from matplotlib.figure import Figure

def wealth_histogram(model):
    n_bins = 15
    colors = ["#66c2a4", "#2ca25f", "#006d2c", "#fc8d59", "#e34a33", "#b30000"]
    labels = ["Designers - Low", "Designers - Mid", "Designers - High", "Producers - Low", "Producers - Mid", "Producers - High"]
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    dwealth_low = [agent.wealth for agent in model.schedule.agents if type(agent) == Designer and agent.hour_fee == 1]
    dwealth_mid = [agent.wealth for agent in model.schedule.agents if type(agent) == Designer and agent.hour_fee == 1.5]
    dwealth_high = [agent.wealth for agent in model.schedule.agents if type(agent) == Designer and agent.hour_fee == 2]
    
    pwealth_low = [agent.wealth for agent in model.schedule.agents if type(agent) == Producer and agent.hour_fee == 1]
    pwealth_mid = [agent.wealth for agent in model.schedule.agents if type(agent) == Producer and agent.hour_fee == 1.5]
    pwealth_high = [agent.wealth for agent in model.schedule.agents if type(agent) == Producer and agent.hour_fee == 2]
    
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    # ax.hist(wealth_vals, bins=10)
    ax.hist([dwealth_low,dwealth_mid,dwealth_high,pwealth_low,pwealth_mid, pwealth_high], 
            n_bins, density=True, histtype='bar', stacked=True, color=colors, label=labels)
    ax.legend(prop={'size': 10})
    ax.set_title('Wealth per type agent')
    solara.FigureMatplotlib(fig)

def work_histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.worked_hours for agent in model.schedule.agents if hasattr(agent,'worked_hours')]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)


In [ ]:
from mesa.experimental import JupyterViz

page = JupyterViz(
    DistMakingModel,
    graphic_params,
    measures=[wealth_histogram,
              ["Designs in Progress",
              "Realized Designs",
              "Products in Progress",
              "On-sale Products",
              "Sold Products"],
              "Resources",
              "Gini"],
    name="Fair Distribution",
    agent_portrayal=agent_portrayal
)
# This is required to render the visualization in the Jupyter notebook
page